In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
import warnings
warnings.filterwarnings("ignore")


In [2]:
df = pd.read_excel(r"C:\Users\kc.hsu\Desktop\0713TBR上課範例.xlsx", None)

In [3]:
df.keys()

dict_keys(['Sellout_daily', '販促價格', '合併客戶', '廠價'])

In [4]:
base = df["Sellout_daily"]

In [5]:
base = base[['出貨點', '實際出貨日', '訂單備註', '產品階層', '訂單類型', '訂單單號', '訂單項次', '買方', '業務員', '大胎業務員', '買方名', '通路', '大胎', '付款方法',
'排序', '物料群組', '物料', '物料說明', '吋別', '中計', '交貨數量', '未稅出貨', '含稅出貨', '發票數量', '發票單價', '發票未稅', '稅額', '發票合計']]

### 販促條件
1. 根據`販促價格`活頁簿的相對應級距來計算TBR價格<font color="red">（可混搭）</font>，和LSR對應的NET價。
2. 客戶總條數可以合併計算
3. 不要<font color="red">**NONO**</font>
4. 大胎契約in [2, 3]，每條可以再扣100元，不是在販促商品內的可以扣200元。只有TBR可以享有。

In [6]:
base = base[base["產品階層"] != "NONO"]

In [7]:
def create_combined_cus_dict(table: pd.DataFrame, columns: list[str]) -> dict:
    """製作一個買方對應合併客戶的字典

    Args:
        table (pd.DataFrame): 將需要的pandas表格丟入
        columns (list[str]): 將需要做key和value的欄位名稱放入列表

    Returns:
        dict: 返回買方-合併客戶字典
    """
    try:
        table = table.applymap(lambda x: x.replace("\xa0", "") if "\xa0" in x else x)
    except:
        pass
    result = dict(zip(table[columns[0]], table[columns[-1]]))
            
    return result

In [8]:
cus_dict = create_combined_cus_dict(df["合併客戶"], ["買方", "合併"])

In [9]:
base["合併客戶"] = base["買方"].map(lambda x: cus_dict[x] if x in cus_dict.keys() else x)

In [10]:
cus_contract_dict = create_combined_cus_dict(base, ["買方", "大胎"])

In [11]:
base["合併客戶契約"] = base["合併客戶"].map(cus_contract_dict)

In [12]:
# 製作販促的字典
prom = df["販促價格"]
tbr_prom = prom[pd.isna(prom["LSR_NET"])]
lsr_prom = prom[pd.notna(prom["LSR_NET"])]
tbr_prom["price_dict"] = tbr_prom[["1-19", "20-49", "50-99", "100-"]].to_dict(orient="record")
tbr_prom_dict = dict(zip(tbr_prom["PCODE"], tbr_prom["price_dict"]))
lsr_prom_dict = dict(zip(lsr_prom["PCODE"], lsr_prom["LSR_NET"]))
price = df["廠價"]
price_dict = dict(zip(price["物料"], price["廠未"]))

In [13]:
# 將TBR拆出來做計算，只有在販促條件的PCODE拿出來做計算
tbr = base[base["物料"].isin(tbr_prom_dict.keys())]

In [14]:
# 計算單一客戶TBR販促商品合計條數
tbr_sum_by_cus = tbr.groupby("買方")[["交貨數量"]].sum().reset_index()
tbr_sum_by_cus_dict = dict(zip(tbr_sum_by_cus["買方"], tbr_sum_by_cus["交貨數量"]))
base["TBR單一客戶販促商品合計數量"] = base.apply(lambda x: tbr_sum_by_cus_dict[x["買方"]] if x["物料"] in tbr_prom_dict.keys() else 0, axis=1)

In [15]:
# 計算合併客戶TBR販促商品合計條數
tbr_prom_sum = tbr.groupby(["合併客戶"])[["交貨數量"]].sum().reset_index()
tbr_prom_sum["級距"] = pd.cut(tbr_prom_sum["交貨數量"], bins=[1, 20, 50, 100, 1000], right=False, labels=["1-19", "20-49", "50-99", "100-"])
tbr_prom_sum_dict = dict(zip(tbr_prom_sum["合併客戶"], tbr_prom_sum["交貨數量"]))

In [16]:
tbr_bins_dict =dict(zip(tbr_prom_sum["合併客戶"], tbr_prom_sum["級距"]))

In [17]:
base["TBR販促商品合計數量"] = base.apply(lambda x: tbr_prom_sum_dict[x["合併客戶"]] if x["物料"] in tbr_prom_dict.keys() else 0, axis=1)

In [18]:
base["級距"] = base.apply(lambda x: tbr_bins_dict[x["合併客戶"]] if x["物料"] in tbr_prom_dict.keys() else "NA", axis=1)

In [19]:
base["TBR販促價格"] = base.apply(lambda x: tbr_prom_dict[x["物料"]][x["級距"]] if x["級距"] != "NA" and x["物料"] in tbr_prom_dict.keys() else 0, axis=1)
base["TBR販促價格"] = base.apply(lambda x: round(x["TBR販促價格"] / 0.965, 0) if x["付款方法"] == "W" else x["TBR販促價格"], axis=1)

In [20]:
base["LSR販促價格"] = base.apply(lambda x: lsr_prom_dict[x["物料"]] if x["物料"] in lsr_prom_dict.keys() and x["付款方法"] != "W" else  round(lsr_prom_dict[x["物料"]] / 0.965, 0) if x["物料"] in lsr_prom_dict.keys() and x["付款方法"] == "W" else 0, axis=1)

In [21]:
base["未稅廠價"] = base["物料"].map(price_dict)

In [22]:
# 計算合併客戶LSR總條數
lsr = base[base["物料"].str.contains("LSR")]
lsr = lsr.groupby("合併客戶")[["交貨數量"]].sum().reset_index()
lsr_volume_dict = dict(zip(lsr["合併客戶"], lsr["交貨數量"]))

In [23]:
base["LSR合併條數"] = base["合併客戶"].map(lsr_volume_dict).fillna(0)

In [24]:
base["TBR折扣後金額"] = base.apply(lambda x: round(x["未稅廠價"] * 0.93 * 0.88, 0) if "TBR" in x["物料"] else 0, axis=1)

In [25]:
base["現金折扣金額"] = base.apply(lambda x: round(x["TBR折扣後金額"] * 0.035, 0) if x["付款方法"] == "C" and "TBR" in x["物料"] else round(x["未稅廠價"] * 0.035, 0), axis=1)

In [26]:
def basic_lsr_discount_amount(df: pd.DataFrame) -> int:
    """計算LSR2的折扣金額

    Args:
        df (pd.DataFrame): Pandas的表格

    Returns:
        int: 返回計算後的折扣金額
    """
    if "LSR" in df["物料"] and 10 <= df["LSR合併條數"] < 20 and df["物料"] not in lsr_prom_dict.keys(): # 折扣金額
        return round((df["未稅廠價"] - df["現金折扣金額"]) * 0.015, 0)
    elif "LSR" in df["物料"] and df["LSR合併條數"] >= 20 and df["物料"] not in lsr_prom_dict.keys():
        return round((df["未稅廠價"] - df["現金折扣金額"]) * 0.035, 0)
    else:
        return 0
        

In [27]:
base["LSR折扣金額"] = base.apply(basic_lsr_discount_amount, axis=1)

In [28]:
# 大胎契約in [2, 3]，每條可以再扣100元，不是在販促商品內的可以扣200元。只有TBR可以享有。
def tbr_add_discount(df: pd.DataFrame) -> int:
    """計算TBR的額外折扣金額

    Args:
        df (pd.DataFrame): Pandas的表格

    Returns:
        int: 返回計算後的單挑折扣金額
    """
    if df["合併客戶契約"] in [2, 3] and "TBR" in df["物料"] and df["物料"] not in tbr_prom_dict.keys():
        return 200
    elif df["合併客戶契約"] in [2, 3] and "TBR" in df["物料"] and df["物料"] in tbr_prom_dict.keys():
        return 100
    else:
        return 0

In [29]:
base["TBR額外折扣"] = base.apply(tbr_add_discount, axis=1)

In [30]:
base["LSR_ZD03"] = base.apply(lambda x: x["未稅廠價"] - x["現金折扣金額"] - x["LSR販促價格"] if "LSR" in x["物料"] and x["LSR販促價格"] != 0 else x["LSR折扣金額"], axis=1)

In [31]:
base["TBR_ZD03"] = base.apply(lambda x: x["未稅廠價"] - x["現金折扣金額"] - x["TBR販促價格"] + x["TBR額外折扣"] if "TBR" in x["物料"] and x["TBR販促價格"] != 0 else x["未稅廠價"] - x["TBR折扣後金額"] + x["TBR額外折扣"] if "TBR" in x["物料"] and x["TBR販促價格"] == 0 else 0, axis=1)

In [32]:
base

,出貨點,實際出貨日,訂單備註,產品階層,訂單類型,訂單單號,訂單項次,買方,業務員,大胎業務員,買方名,通路,大胎,付款方法,排序,物料群組,物料,物料說明,吋別,中計,交貨數量,未稅出貨,含稅出貨,發票數量,發票單價,發票未稅,稅額,發票合計,合併客戶,合併客戶契約,TBR單一客戶販促商品合計數量,TBR販促商品合計數量,級距,TBR販促價格,LSR販促價格,未稅廠價,LSR合併條數,TBR折扣後金額,現金折扣金額,LSR折扣金額,TBR額外折扣,LSR_ZD03,TBR_ZD03
0,FF11,2023-07-03,NaN,NaN,ZSO1,104520036,10,C1B10341,D15,D08,全皇,H,1,C,4,TLSRJ,LSR03459,9.5R17.5 129M R294Z 日,175,LSR2,3,23709,24894,0,0,0,0,0,C1B10341,1,0,0,NA,0.0,7400.0,8190,7.0,0.0,287.0,0.0,0,503.0,0.0
1,FF11,2023-07-03,NaN,NaN,ZSO1,104520036,20,C1B10341,D15,D08,全皇,H,1,C,4,TLSRJ,LSR03733,215/75R175 124L R294Z 日,175,LSR2,4,27940,29337,0,0,0,0,0,C1B10341,1,0,0,NA,0.0,6300.0,7238,7.0,0.0,253.0,0.0,0,685.0,0.0
2,FF11,2023-07-03,送:旭輝_100302_080 (六),NaN,ZSO1,104520153,10,C2B10875,D08,D08,健國,H,3,C,1,TTBR,TBR05553,295/80R225 152M R150Z 日,225,TBR3,1,15912,16708,0,0,0,0,0,C2B10875,3,0,0,NA,0.0,0.0,16381,0.0,13406.0,469.0,0.0,200,0.0,3175.0
3,FF11,2023-07-03,送:順偉_100312_080,NaN,ZSO1,104520185,10,F1F10002,M03,M05,山崎,H,1,C,1,TTBR,TBR0J600,315/80R225 B54M 18 R150Z 泰,225,TBR3,1,16513,17339,0,0,0,0,0,F1F10002,1,1,1,1-19,11900.0,0.0,17000,5.0,13913.0,487.0,0.0,0,0.0,4613.0
4,FF11,2023-07-03,送 達烽輪胎100308 _080_LA,NaN,ZSO1,104520526,10,C1C10259,D08,D08,華聯-五股,C,1,C,1,TTBR,TBR0J136,315/80R225 B54M R294Z 泰,225,TBR3,1,16610,17441,0,0,0,0,0,C2A10568,2,2,23,20-49,12000.0,0.0,17100,24.0,13995.0,490.0,0.0,100,0.0,4710.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,FF73,2023-07-13,NaN,NaN,ZSO1,104526647,10,K1K10076,T04,T04,昇億,H,1,C,4,TLSRJ,LSR03733,215/75R175 124L R294Z 日,175,LSR2,1,6985,7334,0,0,0,0,0,K1K10076,1,0,0,NA,0.0,6300.0,7238,5.0,0.0,253.0,0.0,0,685.0,0.0
253,FF73,2023-07-13,NaN,NaN,ZSO1,104526647,20,K1K10076,T04,T04,昇億,H,1,C,4,TLSRJ,LSR01180,750 R 16 14P R225Z 日,16,LSR2,4,24264,25477,0,0,0,0,0,K1K10076,1,0,0,NA,0.0,5560.0,6286,5.0,0.0,220.0,0.0,0,506.0,0.0
254,FF73,2023-07-13,NaN,NaN,ZSO1,104526806,20,K1K10239,T04,T04,詠靖(弘展),H,1,C,4,TLSRJ,LSR03733,215/75R175 124L R294Z 日,175,LSR2,2,13970,14669,0,0,0,0,0,K1K10239,1,0,0,NA,0.0,6300.0,7238,6.0,0.0,253.0,0.0,0,685.0,0.0
255,FF73,2023-07-13,NaN,NaN,ZSO1,104526806,30,K1K10239,T04,T04,詠靖(弘展),H,1,C,4,TLSRJ,LSR03765,235/75R 175 R294Z 132M 日,175,LSR2,4,28676,30109,0,0,0,0,0,K1K10239,1,0,0,NA,0.0,6670.0,7429,6.0,0.0,260.0,0.0,0,499.0,0.0


In [33]:
sales_status = base[["合併客戶", "買方名", "大胎業務員", "TBR單一客戶販促商品合計數量", "TBR販促商品合計數量", "級距"]].drop_duplicates()

In [34]:
sales_status = sales_status[sales_status["TBR販促商品合計數量"] != 0]

In [35]:
sales_status

,合併客戶,買方名,大胎業務員,TBR單一客戶販促商品合計數量,TBR販促商品合計數量,級距
3,F1F10002,山崎,M05,1,1,1-19
4,C2A10568,華聯-五股,D08,2,23,20-49
7,BGP02000,豐正,C18,6,6,1-19
9,C2A10568,百樂達,D08,7,23,20-49
10,C2A10568,華聯-八里,D08,3,23,20-49
...,...,...,...,...,...,...
199,L1L1A012,允勝,U02,11,13,1-19
201,L1L1F029,順揚,U02,10,10,1-19
206,L1L1H024,信興,U02,31,31,20-49
231,L1L1D068,東昇輪胎,U02,4,4,1-19


In [36]:
with pd.ExcelWriter(r"C:\Users\kc.hsu\Desktop\業務員成績單.xlsx") as writer:
    for sales in sales_status["大胎業務員"].unique():
        df = sales_status[sales_status["大胎業務員"] == sales]
        df.to_excel(writer, sheet_name=sales, index=False)

In [44]:
shops = pd.read_csv(r"C:\Users\kc.hsu\Downloads\STANDARD_aqG6x_BFPTTC店家調查_202307180349_64b60bc1717ed.csv")

In [49]:
shops.columns

Index(['店家名稱', '店家外觀照片', '店家外觀照片 - 儲存位置', '店內及工作區照片', '店內及工作區照片 - 儲存位置',
       '服務車照片', '服務車照片 - 儲存位置', '是否有定位機', '技師人數', '填答時間', '填答秒數', 'IP紀錄',
       '額滿結束註記', '使用者紀錄', '會員時間', 'Hash', '會員編號', '自訂ID', '備註'],
      dtype='object')

In [52]:
shops[["店家外觀照片 - 儲存位置", "店內及工作區照片 - 儲存位置", "服務車照片 - 儲存位置"]] = shops[["店家外觀照片 - 儲存位置", "店內及工作區照片 - 儲存位置", "服務車照片 - 儲存位置"]].applymap(lambda x: x.split("&")[0])

In [71]:
shops.to_csv(r"D:\kc.hsu\OneDrive - Bridgestone\行銷推廣\掛牌店\20230718_FMC問卷回覆.csv", index=False, encoding="utf-8-sig")